# Isotonic Calibration

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.calibration import IsotonicRegression
from sklearn.metrics import fbeta_score, roc_auc_score, average_precision_score, balanced_accuracy_score, confusion_matrix, precision_recall_curve, auc
from sklearn.model_selection import KFold

# =============================================================================
# SCRIPT CONFIGURATION
# =============================================================================
# --- Analysis Mode ---
# List of all possible data sources to calibrate
ALL_DATA_SOURCES = ["preop", "intraop", "combined"]

# --- Paths and Naming ---
RESULTS_DIR = '/home/server/Projects/data/AKI/results/'
# The script will load from e.g., 'tabular_preop_test.pkl'
# and save to 'tabular_preop_test_calibrated.pkl'

# --- Model Name Translations (ensure consistency) ---
model_translate = {
    'base': 'base', 'base_54k': 'base_54k',
    'log_reg': 'Logistic Regression', 'xgb': 'GBT',
    'svm': 'SVM (Linear)',
    'mlp': 'MLP', 'rf': 'Random Forest',
    'knn': 'KNN', 'asa_rule': 'ASA Rule', 'autogluon': 'AutoGluon',
    'lstm': 'LSTM', 'hybrid': 'Hybrid (MLP + LSTM)'
}

# --- Base Model Mapping ---
model_base_map = {
    'LSTM': 'base_54k',
    'Hybrid (MLP + LSTM)': 'base_54k'
}
DEFAULT_BASE = 'base'

# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def find_optimal_f2_threshold(y_true, y_prob):
    """
    Finds the optimal probability threshold to maximize the F2-score.
    """
    thresholds = np.linspace(0.01, 0.99, 200)
    f2_scores = [fbeta_score(y_true, y_prob >= t, beta=2) for t in thresholds]
    optimal_idx = np.argmax(f2_scores)
    return thresholds[optimal_idx]

def recompute_metrics_from_binary(y_true_folds, y_pred_binary_folds):
    """
    Re-computes performance metrics from lists of true labels and binary predictions.
    """
    metrics = {
        'balanced_accuracy': [], 'recall': [], 'precision': [],
        'specificity': [], 'f1': []
    }
    
    for y_true, y_pred in zip(y_true_folds, y_pred_binary_folds):
        if len(np.unique(y_true)) < 2: # Handle folds with only one class
            tn, fp, fn, tp = 0, 0, 0, 0
            if np.unique(y_true)[0] == 0:
                tn = len(y_true)
                fp = np.sum(y_pred)
            else:
                tp = len(y_true)
                fn = len(y_true) - np.sum(y_pred)
        else:
             tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        
        metrics['recall'].append(tp / (tp + fn) if (tp + fn) > 0 else 0.0)
        metrics['specificity'].append(tn / (tn + fp) if (tn + fp) > 0 else 0.0)
        metrics['precision'].append(tp / (tp + fp) if (tp + fp) > 0 else 0.0)
        metrics['f1'].append(fbeta_score(y_true, y_pred, beta=1))
        metrics['balanced_accuracy'].append(balanced_accuracy_score(y_true, y_pred))
        
    return metrics

# =============================================================================
# MAIN CALIBRATION AND EVALUATION FUNCTION
# =============================================================================

def perform_calibration_and_evaluation(data_source):
    """
    Orchestrates the entire calibration process for a given data source.
    """
    print(f"\n{'~'*25} CALIBRATING: {data_source.upper()} {'~'*25}\n")
    
    # Define input and output file paths
    input_pkl = os.path.join(RESULTS_DIR, f"tabular_{data_source}_test.pkl")
    output_pkl = os.path.join(RESULTS_DIR, f"tabular_{data_source}_test_calibrated.pkl")

    try:
        df_original = pd.read_pickle(input_pkl)
        # Apply model name translations
        df_original['model_name'] = df_original['model_name'].map(model_translate)
        df_original.dropna(subset=['model_name'], inplace=True)
        print(f"Successfully loaded original results from: {input_pkl}")
    except FileNotFoundError:
        print(f"ERROR: Original results file not found at {input_pkl}. Skipping.")
        return

    # Prepare to store calibrated results
    calibrated_results_list = []

    # Separate base models (ground truth) from predictive models
    df_base = df_original[df_original['model_name'].str.contains('base', na=False)].copy()
    df_models = df_original[~df_original['model_name'].str.contains('base', na=False)].copy()

    # --- Process each predictive model ---
    for _, model_row in df_models.iterrows():
        model_name = model_row['model_name']
        print(f"  -> Processing model: {model_name}")

        # Determine which base model's ground truth to use
        base_to_use = model_base_map.get(model_name, DEFAULT_BASE)
        try:
            y_true_folds = df_base[df_base['model_name'] == base_to_use]['y_pred_binary'].iloc[0]
            y_prob_folds = model_row['y_prob']
        except IndexError:
            print(f"     WARNING: Base model '{base_to_use}' not found for '{model_name}'. Skipping.")
            continue

        # Concatenate fold data into single long vectors for calibration
        y_true_flat = np.concatenate(y_true_folds)
        y_prob_flat = np.concatenate(y_prob_folds)
        
        # Store original fold lengths to split data back later
        fold_lengths = [len(fold) for fold in y_true_folds]
        
        # --- Calibrate Probabilities using Isotonic Regression with manual CV ---
        calibrated_y_prob_flat = np.zeros_like(y_prob_flat)
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        for train_idx, test_idx in kf.split(y_prob_flat):
            isotonic_calibrator = IsotonicRegression(out_of_bounds='clip')
            isotonic_calibrator.fit(y_prob_flat[train_idx], y_true_flat[train_idx])
            calibrated_y_prob_flat[test_idx] = isotonic_calibrator.predict(y_prob_flat[test_idx])

        # --- Find new threshold on calibrated probabilities ---
        new_threshold = find_optimal_f2_threshold(y_true_flat, calibrated_y_prob_flat)
        print(f"     New F2-optimal threshold: {new_threshold:.4f}")

        # --- Generate new binary predictions ---
        new_y_pred_binary_flat = (calibrated_y_prob_flat >= new_threshold).astype(int)

        # --- Split flat arrays back into fold structure ---
        cursor = 0
        new_y_pred_binary_folds = []
        calibrated_y_prob_folds = []
        for length in fold_lengths:
            new_y_pred_binary_folds.append(new_y_pred_binary_flat[cursor : cursor + length])
            calibrated_y_prob_folds.append(calibrated_y_prob_flat[cursor : cursor + length])
            cursor += length

        # --- Re-compute threshold-dependent metrics ---
        new_metrics = recompute_metrics_from_binary(y_true_folds, new_y_pred_binary_folds)
        
        # --- MODIFICATION: Calculate AUPRC per-fold from calibrated probabilities ---
        auprc_scores = []
        num_folds_to_process = min(len(y_true_folds), len(calibrated_y_prob_folds))
        if num_folds_to_process < len(calibrated_y_prob_folds):
            print(f"     INFO [{model_name}]: Mismatched fold counts for AUPRC. Processing {num_folds_to_process} matching folds.")
        for i in range(num_folds_to_process):
            if len(y_true_folds[i]) == len(calibrated_y_prob_folds[i]):
                precision, recall, _ = precision_recall_curve(y_true_folds[i], calibrated_y_prob_folds[i])
                auprc_scores.append(auc(recall, precision))
        # --- END MODIFICATION ---

        # Create a new row for the calibrated results DataFrame
        new_row = model_row.copy()
        
        # Update the row with new data
        new_row['y_prob'] = calibrated_y_prob_folds
        new_row['y_pred_binary'] = new_y_pred_binary_folds
        new_row['threshold'] = new_threshold
        
        for metric_name, values in new_metrics.items():
            new_row[metric_name] = values
        
        # Add the newly calculated AUPRC scores
        new_row['pr_auc'] = auprc_scores if auprc_scores else None
            
        calibrated_results_list.append(new_row)

    # Combine calibrated models and original base models into one DataFrame
    calibrated_df = pd.concat([pd.DataFrame(calibrated_results_list), df_base], ignore_index=True)

    # Save the calibrated results to a new pickle file
    with open(output_pkl, 'wb') as f:
        pickle.dump(calibrated_df, f)
    print(f"\nSuccessfully saved calibrated results to: {output_pkl}")
    print(f"{'~'*25} CALIBRATION COMPLETE: {data_source.upper()} {'~'*25}\n")


# =============================================================================
# MAIN EXECUTION BLOCK
# =============================================================================
if __name__ == "__main__":
    for source in ALL_DATA_SOURCES:
        perform_calibration_and_evaluation(source)



~~~~~~~~~~~~~~~~~~~~~~~~~ CALIBRATING: PREOP ~~~~~~~~~~~~~~~~~~~~~~~~~

Successfully loaded original results from: /home/server/Projects/data/AKI/results/tabular_preop_test.pkl
  -> Processing model: Logistic Regression
     New F2-optimal threshold: 0.1430
  -> Processing model: GBT
     New F2-optimal threshold: 0.1183
  -> Processing model: Random Forest
     New F2-optimal threshold: 0.1331
  -> Processing model: SVM (Linear)
     New F2-optimal threshold: 0.1183
  -> Processing model: MLP
     New F2-optimal threshold: 0.1134
  -> Processing model: KNN
     New F2-optimal threshold: 0.1134
  -> Processing model: ASA Rule
     New F2-optimal threshold: 0.0297
  -> Processing model: AutoGluon
     New F2-optimal threshold: 0.1331

Successfully saved calibrated results to: /home/server/Projects/data/AKI/results/tabular_preop_test_calibrated.pkl
~~~~~~~~~~~~~~~~~~~~~~~~~ CALIBRATION COMPLETE: PREOP ~~~~~~~~~~~~~~~~~~~~~~~~~


~~~~~~~~~~~~~~~~~~~~~~~~~ CALIBRATING: INTRAOP ~~~~~~~~~~~

# Figure Creation

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.calibration import calibration_curve
import os

# =============================================================================
# SCRIPT CONFIGURATION
# =============================================================================
# --- Analysis Mode ---
# Set to True to run for all sources in ALL_DATA_SOURCES.
# Set to False to run only for the single source specified in DATA_SOURCE.
ANALYZE_ALL_SOURCES = True

# --- NEW: Plotting Options ---
# Set to True to display shaded confidence intervals on ROC and PR curves.
PLOT_CONFIDENCE_INTERVALS = True

# --- Select which dataset's results you want to analyze (if ANALYZE_ALL_SOURCES is False) ---
DATA_SOURCE = "intraop"

# --- List of all possible data sources (used when ANALYZE_ALL_SOURCES is True) ---
ALL_DATA_SOURCES = ["preop", "intraop", "combined"]

# --- Paths and Naming ---
RESULTS_DIR = '/home/server/Projects/data/AKI/results/'
FIGURES_DIR = 'figures/curves'  # Define the folder to save plots

# Note: The full path will be constructed dynamically in the analysis function.

# --- Model Name Translations for Plots ---
model_translate = {
    'base': 'base', 'base_54k': 'base_54k', # Add any new base models here
    'log_reg': 'Logistic Regression', 'xgb': 'GBT',
    'svm': 'SVM (Linear)',  # MODIFIED: Changed 'svm_linear' to 'svm' to match first script
    'mlp': 'MLP', 'rf': 'Random Forest',
    'knn': 'KNN', 'asa_rule': 'ASA Rule', 'autogluon': 'AutoGluon',
    'lstm': 'LSTM', 'hybrid': 'Hybrid (MLP + LSTM)'
}

# Define which ground-truth 'base' model to use for specific predictive models.
# This allows comparing models trained on different underlying datasets.
# Models not listed here will use the DEFAULT_BASE.
model_base_map = {
    'LSTM': 'base_54k',
    'Hybrid (MLP + LSTM)': 'base_54k'
}
DEFAULT_BASE = 'base'


# --- Consistent Color Mapping for Models ---
model_colors = {
    'Logistic Regression': 'green',
    'GBT': 'red',
    'SVM (Linear)': 'purple', # This entry is already correct
    'MLP': 'brown',
    'Random Forest': 'crimson',
    'KNN': 'gray',
    'ASA Rule': 'orange',
    'AutoGluon': 'blue',
    'LSTM': 'darkorange',
    'Hybrid (MLP + LSTM)': 'teal'
}


# =============================================================================
# PLOTTING FUNCTIONS (ADAPTED FOR NEW .pkl FORMAT)
# =============================================================================

def plot_roc_curves(df, data_source_name, save_dir):
    """Plots the mean ROC curve for each model, optionally with a shaded confidence interval."""
    plt.figure(figsize=(10, 8), dpi=300)
    
    # Load all available 'base' models' ground-truth data
    base_names = [name for name in df['model_name'].unique() if 'base' in name]
    if not base_names:
        print("ERROR: No 'base' model with true labels found. Cannot plot curves.")
        return
        
    base_models_data = {
        name: df[df['model_name'] == name]['y_pred_binary'].values[0]
        for name in base_names
    }
    
    df_to_plot = df[~df['model_name'].str.contains('base')]
    
    # Pre-calculate AUCs to sort the legend
    model_performance = []
    for index, model_row in df_to_plot.iterrows():
        model_name = model_row['model_name']
        
        # Determine which base model's ground truth to use
        base_to_use = model_base_map.get(model_name, DEFAULT_BASE)
        if base_to_use not in base_models_data:
            print(f"WARNING [ROC]: Base model '{base_to_use}' for model '{model_name}' not found. Skipping.")
            continue
        y_true_runs = base_models_data[base_to_use]

        aucs = []
        for i in range(len(model_row['y_prob'])):
            if i >= len(y_true_runs) or len(y_true_runs[i]) != len(model_row['y_prob'][i]):
                print(f"WARNING [ROC]: Skipping fold {i} for model '{model_name}'. Inconsistent sample numbers.")
                continue
            fpr, tpr, _ = roc_curve(y_true_runs[i], model_row['y_prob'][i])
            aucs.append(auc(fpr, tpr))

        if not aucs:
            print(f"WARNING [ROC]: Could not calculate AUC for model '{model_name}'. Skipping.")
            continue
            
        model_performance.append((model_name, np.mean(aucs), model_row))
    
    model_performance.sort(key=lambda x: x[1], reverse=True)

    for model_name, mean_auc, model_row in model_performance:
        base_to_use = model_base_map.get(model_name, DEFAULT_BASE)
        y_true_runs = base_models_data[base_to_use]
        y_prob_runs = model_row['y_prob']
        
        mean_fpr = np.linspace(0, 1, 100)
        tprs = []
        for i in range(len(y_prob_runs)):
            if i >= len(y_true_runs) or len(y_true_runs[i]) != len(y_prob_runs[i]):
                continue
            fpr, tpr, _ = roc_curve(y_true_runs[i], y_prob_runs[i])
            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
        
        if not tprs: continue
            
        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        color = model_colors.get(model_name, None)
        
        # --- MODIFIED: Conditionally plot confidence interval ---
        if PLOT_CONFIDENCE_INTERVALS:
            # The shaded region represents +/- 1 standard deviation from the mean TPR.
            std_tpr = np.std(tprs, axis=0)
            tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
            tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
            plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color=color, alpha=0.2)
        # --- END MODIFIED ---

        valid_aucs = [auc(roc_curve(y_true_runs[i], y_prob_runs[i])[0], roc_curve(y_true_runs[i], y_prob_runs[i])[1]) for i in range(len(y_prob_runs)) if i < len(y_true_runs) and len(y_true_runs[i]) == len(y_prob_runs[i])]
        std_auc = np.std(valid_aucs)
        
        plt.plot(mean_fpr, mean_tpr, color=color, label=f'{model_name} (AUC = {mean_auc:.2f} \u00B1 {std_auc:.2f})', lw=2)

    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Chance')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Mean ROC Curves on {data_source_name.upper()} Data')
    plt.legend(loc='lower right', fontsize='large')
    plt.grid(True)
    
    # Save the figure
    save_path = os.path.join(save_dir, f'roc_curves_{data_source_name}.png')
    plt.savefig(save_path, bbox_inches='tight')
    print(f"Saved ROC curve plot to: {save_path}")
    plt.close() # Close the plot to free memory

def plot_pr_curves(df, data_source_name, save_dir):
    """Plots the mean Precision-Recall curve for each model, optionally with a shaded confidence interval."""
    plt.figure(figsize=(10, 8), dpi=300)
    
    base_names = [name for name in df['model_name'].unique() if 'base' in name]
    if not base_names:
        print("ERROR: No 'base' model with true labels found. Cannot plot curves.")
        return
        
    base_models_data = {
        name: df[df['model_name'] == name]['y_pred_binary'].values[0]
        for name in base_names
    }
    
    df_to_plot = df[~df['model_name'].str.contains('base')]
    
    model_performance = []
    for index, model_row in df_to_plot.iterrows():
        model_name = model_row['model_name']
        base_to_use = model_base_map.get(model_name, DEFAULT_BASE)
        if base_to_use not in base_models_data:
            print(f"WARNING [PR]: Base model '{base_to_use}' for model '{model_name}' not found. Skipping.")
            continue
        y_true_runs = base_models_data[base_to_use]
        
        pr_aucs = []
        for i in range(len(model_row['y_prob'])):
            if i >= len(y_true_runs) or len(y_true_runs[i]) != len(model_row['y_prob'][i]):
                print(f"WARNING [PR]: Skipping fold {i} for model '{model_name}'. Inconsistent sample numbers.")
                continue
            precision, recall, _ = precision_recall_curve(y_true_runs[i], model_row['y_prob'][i])
            pr_aucs.append(auc(recall, precision))

        if not pr_aucs:
            print(f"WARNING [PR]: Could not calculate AUC for model '{model_name}'. Skipping.")
            continue
            
        model_performance.append((model_row['model_name'], np.mean(pr_aucs), model_row))

    model_performance.sort(key=lambda x: x[1], reverse=True)

    for model_name, mean_pr_auc, model_row in model_performance:
        base_to_use = model_base_map.get(model_name, DEFAULT_BASE)
        y_true_runs = base_models_data[base_to_use]
        y_prob_runs = model_row['y_prob']
        
        mean_recall = np.linspace(0, 1, 100)
        precisions = []
        for i in range(len(y_prob_runs)):
            if i >= len(y_true_runs) or len(y_true_runs[i]) != len(y_prob_runs[i]):
                continue
            precision, recall, _ = precision_recall_curve(y_true_runs[i], y_prob_runs[i])
            # Interpolate precision values for a common recall axis
            interp_prec = np.interp(mean_recall, recall[::-1], precision[::-1])
            precisions.append(interp_prec)

        if not precisions: continue

        mean_prec = np.mean(precisions, axis=0)
        color = model_colors.get(model_name, None)
        
        # --- MODIFIED: Conditionally plot confidence interval ---
        if PLOT_CONFIDENCE_INTERVALS:
            # The shaded region represents +/- 1 standard deviation from the mean precision.
            std_prec = np.std(precisions, axis=0)
            prec_upper = np.minimum(mean_prec + std_prec, 1)
            prec_lower = np.maximum(mean_prec - std_prec, 0)
            plt.fill_between(mean_recall, prec_lower, prec_upper, color=color, alpha=0.2)
        # --- END MODIFIED ---

        valid_pr_aucs = [auc(precision_recall_curve(y_true_runs[i], y_prob_runs[i])[1], precision_recall_curve(y_true_runs[i], y_prob_runs[i])[0]) for i in range(len(y_prob_runs)) if i < len(y_true_runs) and len(y_true_runs[i]) == len(y_prob_runs[i])]
        std_pr_auc = np.std(valid_pr_aucs)

        plt.plot(mean_recall, mean_prec, color=color, label=f'{model_name} (AUC = {mean_pr_auc:.2f} \u00B1 {std_pr_auc:.2f})', lw=2)

    # Use the default base for the 'no skill' line as a general reference
    y_true_default_runs = base_models_data[DEFAULT_BASE]
    y_true_default_flat = np.concatenate(y_true_default_runs)
    no_skill = len(y_true_default_flat[y_true_default_flat==1]) / len(y_true_default_flat)
    plt.plot([0, 1], [no_skill, no_skill], linestyle='--', color='gray', label='No Skill')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Mean Precision-Recall Curves on {data_source_name.upper()} Data')
    plt.legend(loc='upper right', fontsize='large')
    plt.grid(True)
    
    # Save the figure
    save_path = os.path.join(save_dir, f'pr_curves_{data_source_name}.png')
    plt.savefig(save_path, bbox_inches='tight')
    print(f"Saved Precision-Recall curve plot to: {save_path}")
    plt.close() # Close the plot to free memory

def plot_calibration_curves(df, data_source_name, save_dir):
    """Plots calibration curves by pooling all predictions and saves the figure."""
    plt.figure(figsize=(10, 8), dpi=300)
    
    base_names = [name for name in df['model_name'].unique() if 'base' in name]
    if not base_names:
        print("ERROR: No 'base' model with true labels found. Cannot plot curves.")
        return
        
    base_models_data = {
        name: df[df['model_name'] == name]['y_pred_binary'].values[0]
        for name in base_names
    }

    df_to_plot = df[~df['model_name'].str.contains('base')]

    for index, model_row in df_to_plot.iterrows():
        model_name = model_row['model_name']
        base_to_use = model_base_map.get(model_name, DEFAULT_BASE)
        if base_to_use not in base_models_data:
            print(f"WARNING [Calibration]: Base model '{base_to_use}' for model '{model_name}' not found. Skipping.")
            continue
            
        y_true_runs = base_models_data[base_to_use]
        y_prob_runs = model_row['y_prob']
        
        # Check for total length consistency before concatenating
        len_y_true = sum(len(arr) for arr in y_true_runs)
        len_y_prob = sum(len(arr) for arr in y_prob_runs)

        if len_y_true != len_y_prob:
            print(f"WARNING [Calibration]: Skipping model '{model_name}'. Pooled predictions have mismatched samples.")
            continue
            
        y_true_all = np.concatenate(y_true_runs)
        y_prob_all = np.concatenate(y_prob_runs)

        prob_true, prob_pred = calibration_curve(y_true_all, y_prob_all, n_bins=15, strategy='uniform')
        plt.plot(prob_pred, prob_true, marker='o', linestyle='-', color=model_colors.get(model_name, None), label=f'{model_name}')

    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfectly Calibrated')
    plt.xlabel('Mean Predicted Probability')
    plt.ylabel('Fraction of Positives')
    plt.title(f'Calibration Curves on {data_source_name.upper()} Data')
    plt.legend(loc='upper left', fontsize='large')
    plt.grid(True)
    
    # Save the figure
    save_path = os.path.join(save_dir, f'calibration_curves_{data_source_name}.png')
    plt.savefig(save_path, bbox_inches='tight')
    print(f"Saved calibration curve plot to: {save_path}")
    plt.close() # Close the plot to free memory

# =============================================================================
# METRICS TABLE FUNCTION
# =============================================================================
def print_metrics_table(df, data_source_name):
    """Prints a formatted table of performance metrics with 95% CIs."""
    df_to_print = df[~df['model_name'].str.contains('base')].copy()
    
    metrics_to_print = ['roc_auc', 'balanced_accuracy', 'recall', 'precision', 'specificity', 'f1']
    
    print("\n" + "="*80)
    print(f"PERFORMANCE METRICS SUMMARY ({data_source_name.upper()})")
    print("="*80)
    print(f"{'Model':<25} {'Metric':<20} {'Mean':<10} {'95% CI Lower':<15} {'95% CI Upper':<15}")
    print("-"*80)

    for index, model_row in df_to_print.iterrows():
        model_name = model_row['model_name']
        print(f"{model_name:<25}")
        for metric in metrics_to_print:
            if metric not in model_row or model_row[metric] is None or len(np.atleast_1d(model_row[metric])) == 0:
                print(f"{'':<25} {metric:<20} {'N/A':<10} {'N/A':<15} {'N/A':<15}")
                continue

            arr = model_row[metric]
            
            if np.isscalar(arr) or len(arr) == 1:
                mean = np.mean(arr)
                ci_lower, ci_upper = "N/A", "N/A"
            else:
                mean = np.mean(arr)
                sem = stats.sem(arr)
                n = len(arr)

                if sem > 0 and n > 1:
                    ci = stats.t.interval(0.95, df=n-1, loc=mean, scale=sem)
                    ci_lower, ci_upper = f"{ci[0]:.4f}", f"{ci[1]:.4f}"
                else:
                    ci_lower, ci_upper = "N/A", "N/A"
            
            print(f"{'':<25} {metric:<20} {mean:<10.4f} {ci_lower:<15} {ci_upper:<15}")
        print("-"*80)

# =============================================================================
# MAIN ANALYSIS EXECUTION
# =============================================================================
def run_full_analysis(data_source):
    """
    Loads data for a given source and runs all plotting and metric functions.
    """
    print(f"\n{'~'*25} RUNNING ANALYSIS FOR: {data_source.upper()} {'~'*25}\n")
    
    # Create the figures directory if it doesn't exist
    os.makedirs(FIGURES_DIR, exist_ok=True)
    
    output_pkl = os.path.join(RESULTS_DIR, f"tabular_{data_source}_test.pkl")

    try:
        df = pd.read_pickle(output_pkl)
        df['model_name'] = df['model_name'].map(model_translate)
        df.dropna(subset=['model_name'], inplace=True)
        print(f"Successfully loaded and processed results from: {output_pkl}")
        found_models = df[~df['model_name'].str.contains('base')]['model_name'].unique().tolist()
        print(f"Found models: {found_models}")
    except FileNotFoundError:
        print(f"ERROR: Results file not found at {output_pkl}. Skipping this data source.")
        return

    # Pass the save directory to the plotting functions
    plot_roc_curves(df, data_source, FIGURES_DIR)
    plot_pr_curves(df, data_source, FIGURES_DIR)
    plot_calibration_curves(df, data_source, FIGURES_DIR)
    
    print_metrics_table(df, data_source)
    print(f"\n{'~'*25} COMPLETED ANALYSIS FOR: {data_source.upper()} {'~'*25}\n")


if __name__ == "__main__":
    if ANALYZE_ALL_SOURCES:
        for source in ALL_DATA_SOURCES:
            run_full_analysis(source)
    else:
        run_full_analysis(DATA_SOURCE)


~~~~~~~~~~~~~~~~~~~~~~~~~ RUNNING ANALYSIS FOR: PREOP ~~~~~~~~~~~~~~~~~~~~~~~~~

Successfully loaded and processed results from: /home/server/Projects/data/AKI/results/tabular_preop_test.pkl
Found models: ['Logistic Regression', 'GBT', 'Random Forest', 'SVM (Linear)', 'MLP', 'KNN', 'ASA Rule', 'AutoGluon']
Saved ROC curve plot to: figures/curves/roc_curves_preop.png
Saved Precision-Recall curve plot to: figures/curves/pr_curves_preop.png
Saved calibration curve plot to: figures/curves/calibration_curves_preop.png

PERFORMANCE METRICS SUMMARY (PREOP)
Model                     Metric               Mean       95% CI Lower    95% CI Upper   
--------------------------------------------------------------------------------
Logistic Regression      
                          roc_auc              0.9122     0.9097          0.9148         
                          balanced_accuracy    0.8410     0.8380          0.8439         
                          recall               0.8043     0.7982  

# Performance Metrics Table

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import os
from sklearn.metrics import precision_recall_curve, auc
import html
import uuid

# =============================================================================
# SCRIPT CONFIGURATION
# =============================================================================
# --- List of data sources to include in the table ---
DATA_SOURCES = ["preop", "intraop", "combined"]

# --- File Paths ---
# IMPORTANT: Update this path to your actual results directory
RESULTS_DIR = '/home/server/Projects/data/AKI/results/'
OUTPUT_FILE = 'performance_table.md'

# --- Model Name Translations for Plots ---
# This ensures consistency with your main analysis script
model_translate = {
    'base': 'base', 'base_54k': 'base_54k',
    'log_reg': 'LR', 'xgb': 'GBT',
    'svm': 'SVM',
    'mlp': 'MLP', 'rf': 'RF',
    'knn': 'KNN', 'asa_rule': 'ASA Rule', 'autogluon': 'AutoGluon',
    'lstm': 'LSTM', 'hybrid': 'MLP+LSTM'
}

# --- Base Model Mapping ---
# Defines which ground-truth 'base' model to use for specific predictive models.
model_base_map = {
    'LSTM': 'base_54k',
    'MLP+LSTM': 'base_54k'
}
DEFAULT_BASE = 'base'


# --- Metrics to include in the table and their display names ---
# The order in this dictionary determines the column order in the table
METRICS_MAP = {
    'roc_auc': 'AUROC',
    'pr_auc': 'AUPRC',
    'recall': 'Sensitivity',
    'specificity': 'Specificity',
    'precision': 'Precision',
    'f1': 'F-score',
    'balanced_accuracy': 'Accuracy'
}

# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def get_metric_ci(metric_array):
    """Calculates the mean and 95% CI for a given array of metric scores."""
    if metric_array is None or pd.isnull(metric_array).all() or len(np.atleast_1d(metric_array)) == 0:
        return "N/A", "N/A"
        
    arr = np.atleast_1d(metric_array)
    
    if np.isscalar(arr) or len(arr) == 1:
        mean_str = f"{np.mean(arr):.3f}"
        ci_str = "N/A"
    else:
        mean = np.mean(arr)
        sem = stats.sem(arr)
        n = len(arr)
        mean_str = f"{mean:.3f}"

        if sem > 0 and n > 1:
            ci = stats.t.interval(0.95, df=n-1, loc=mean, scale=sem)
            ci_str = f"({ci[0]:.3f}, {ci[1]:.3f})"
        else:
            ci_str = "N/A"
            
    return mean_str, ci_str

# =============================================================================
# MAIN SCRIPT LOGIC
# =============================================================================

def generate_performance_tables():
    """
    Generates and displays performance tables in both Markdown and rich-text HTML format,
    with inline styles for easy copy-pasting to Google Docs.
    """
    all_results = {}
    base_models_data = {}

    # --- 1. Load data from all specified sources ---
    for source in DATA_SOURCES:
        pkl_path = os.path.join(RESULTS_DIR, f"tabular_{source}_test.pkl")
        try:
            df = pd.read_pickle(pkl_path)
            df['model_name'] = df['model_name'].map(model_translate)
            df.dropna(subset=['model_name'], inplace=True)
            
            df_models = df[~df['model_name'].str.contains('base', na=False)].copy()
            df_base = df[df['model_name'].str.contains('base', na=False)].copy()
            
            all_results[source] = df_models
            
            for _, base_row in df_base.iterrows():
                # Assuming 'y_pred_binary' for base models holds the true labels for each run
                base_models_data[base_row['model_name']] = base_row['y_pred_binary']
                
            print(f"Successfully loaded data for: {source.upper()}")
        except FileNotFoundError:
            print(f"WARNING: Results file not found for '{source}'. Skipping.")
            continue

    # --- 2. Initialize Markdown and define HTML table styles ---
    # --- Markdown Initialization ---
    markdown_lines = []
    md_header = "| Model | " + " | ".join(METRICS_MAP.values()) + " |"
    md_separator = "|" + "---|" * (len(METRICS_MAP) + 1)
    markdown_lines.extend([md_header, md_separator])

    # --- HTML Styling for Google Docs ---
    table_style = "font-family: 'Times New Roman', Times, serif; font-size: 8pt; border-collapse: collapse; width: 100%; border: 1px solid #999;"
    th_style = "border: 1px solid #999; padding: 6px; text-align: center; background-color: #f2f2f2; font-weight: bold;"
    section_header_style = "border: 1px solid #999; padding: 6px; text-align: left; background-color: #e0e0e0; font-weight: bold;"
    model_name_style = "border: 1px solid #999; padding: 6px; text-align: left; font-weight: bold;"
    cell_style = "border: 1px solid #999; padding: 6px; text-align: center;"
    ci_style = "color: #444;" # Style for the confidence interval text

    # --- HTML Initialization ---
    html_lines = [f'<table style="{table_style}">', '<thead>', '<tr>']
    html_lines.append(f'<th style="{th_style}">Model</th>')
    for name in METRICS_MAP.values():
        html_lines.append(f'<th style="{th_style}">{name}</th>')
    html_lines.extend(['</tr>', '</thead>', '<tbody>'])


    # --- 3. Populate tables section by section ---
    for source_name, df_data in all_results.items():
        # Add section titles
        section_title_text = f"**{source_name.replace('_', ' ').title()} Data**"
        markdown_lines.append(f"| {section_title_text} |" + " |" * len(METRICS_MAP))
        html_lines.append(f'<tr><td colspan="{len(METRICS_MAP)+1}" style="{section_header_style}">{section_title_text.replace("**","")}</td></tr>')

        df_data = df_data.sort_values(by='model_name')

        for _, model_row in df_data.iterrows():
            model_name = model_row['model_name']
            
            # --- Calculate metrics ---
            final_metrics = {key: model_row.get(key) for key in METRICS_MAP.keys()}
            if 'pr_auc' in final_metrics and (final_metrics['pr_auc'] is None or np.all(pd.isnull(final_metrics['pr_auc']))):
                auprc_scores = []
                base_to_use = model_base_map.get(model_name, DEFAULT_BASE)
                if base_to_use in base_models_data:
                    y_true_runs = base_models_data[base_to_use]
                    y_prob_runs = model_row.get('y_prob')
                    if y_prob_runs is not None:
                        for i in range(len(y_prob_runs)):
                            if i < len(y_true_runs) and len(y_true_runs[i]) == len(y_prob_runs[i]):
                                precision, recall, _ = precision_recall_curve(y_true_runs[i], y_prob_runs[i])
                                auprc_scores.append(auc(recall, precision))
                final_metrics['pr_auc'] = auprc_scores if auprc_scores else None

            # --- Build Markdown Rows ---
            md_mean_values = [model_name]
            md_ci_values = [""]
            
            # --- Build HTML Row ---
            html_row_cells = [f'<td style="{model_name_style}">{model_name}</td>']

            for metric_key in METRICS_MAP.keys():
                mean_str, ci_str = get_metric_ci(final_metrics.get(metric_key))
                
                md_mean_values.append(mean_str)
                md_ci_values.append(ci_str)
                
                # HTML cell with inline styles
                html_cell = f'{mean_str}<br><span style="{ci_style}">{ci_str}</span>' if ci_str != "N/A" else mean_str
                html_row_cells.append(f'<td style="{cell_style}">{html_cell}</td>')

            markdown_lines.append("| " + " | ".join(md_mean_values) + " |")
            markdown_lines.append("| " + " | ".join(md_ci_values) + " |")
            html_lines.append("<tr>" + "".join(html_row_cells) + "</tr>")
    
    html_lines.append("</tbody></table>")
    
    # --- 4. Finalize and Output ---
    # --- Markdown Output ---
    final_markdown = "\n".join(markdown_lines)
    try:
        with open(OUTPUT_FILE, 'w') as f:
            f.write(final_markdown)
        print(f"\nSuccessfully saved Markdown table to: {OUTPUT_FILE}")
    except IOError as e:
        print(f"\nERROR: Could not save Markdown file. {e}")
        
    print("\n--- Generated Markdown Table ---\n")
    print(final_markdown)
    
    # --- HTML Output (for Jupyter) ---
    table_html = "\n".join(html_lines)
    
    collapsible_id = f"collapsible-{uuid.uuid4().hex}"
    escaped_html = html.escape(table_html)
    
    full_html_output = f"""
    <div>
        {table_html}
        <br>
        <a href="#" onclick="document.getElementById('{collapsible_id}').style.display = document.getElementById('{collapsible_id}').style.display === 'none' ? 'block' : 'none'; return false;">
            Show/Hide HTML Source
        </a>
        <pre id="{collapsible_id}" style="display:none; background-color:#f8f8f8; border:1px solid #ccc; padding:10px; white-space: pre-wrap; word-wrap: break-word;"><code>{escaped_html}</code></pre>
    </div>
    """
    
    try:
        from IPython.display import display, HTML
        print("\n--- Rich Text Table (for Jupyter Notebook) ---")
        display(HTML(full_html_output))
    except ImportError:
        print("\n--- HTML Output (for non-Jupyter environments) ---")
        print("To see the rich text table, run this script in a Jupyter Notebook or open the saved HTML file.")
        try:
            html_file_name = OUTPUT_FILE.replace('.md', '.html')
            with open(html_file_name, 'w', encoding='utf-8') as f:
                f.write(full_html_output)
            print(f"HTML table saved to: {html_file_name}")
        except IOError as e:
                print(f"\nERROR: Could not save HTML file. {e}")


if __name__ == "__main__":
    generate_performance_tables()

Successfully loaded data for: PREOP
Successfully loaded data for: INTRAOP
Successfully loaded data for: COMBINED

Successfully saved Markdown table to: performance_table.md

--- Generated Markdown Table ---

| Model | AUROC | AUPRC | Sensitivity | Specificity | Precision | F-score | Accuracy |
|---|---|---|---|---|---|---|---|
| **Preop Data** | | | | | | | |
| ASA Rule | 0.758 | 0.192 | 0.067 | 0.992 | 0.295 | 0.109 | 0.530 |
|  | (0.755, 0.761) | (0.185, 0.199) | (0.063, 0.071) | (0.992, 0.993) | (0.280, 0.310) | (0.103, 0.116) | (0.528, 0.532) |
| AutoGluon | 0.931 | 0.625 | 0.819 | 0.887 | 0.254 | 0.388 | 0.853 |
|  | (0.928, 0.933) | (0.618, 0.631) | (0.811, 0.828) | (0.883, 0.890) | (0.250, 0.259) | (0.383, 0.393) | (0.850, 0.856) |
| GBT | 0.926 | 0.614 | 0.626 | 0.972 | 0.512 | 0.564 | 0.799 |
|  | (0.924, 0.928) | (0.609, 0.620) | (0.620, 0.633) | (0.971, 0.973) | (0.506, 0.518) | (0.559, 0.568) | (0.796, 0.802) |
| KNN | 0.909 | 0.542 | 0.126 | 1.000 | 0.949 | 0.223 | 0.563 |

# Reclassification Analysis

In [11]:
import pandas as pd
import numpy as np
import os
import html
from IPython.display import display, HTML
import scipy.stats as stats

# =============================================================================
# SCRIPT CONFIGURATION
# =============================================================================
# --- Analysis Settings ---
N_FOLDS = 25 # Number of cross-validation folds

# --- Paths and Naming ---
RESULTS_DIR = '/home/server/Projects/data/AKI/results/'

# --- Model Name Translations ---
model_translate = {
    'base': 'base', 'base_54k': 'base_54k',
    'log_reg': 'Logistic Regression', 'xgb': 'GBT',
    'svm': 'SVM (Linear)',
    'mlp': 'MLP', 'rf': 'Random Forest',
    'knn': 'KNN', 'asa_rule': 'ASA Rule', 'autogluon': 'AutoGluon',
    'lstm': 'LSTM', 'hybrid': 'Hybrid (MLP + LSTM)'
}

# --- Base Model Mapping ---
model_base_map = {
    'LSTM': 'base_54k',
    'Hybrid (MLP + LSTM)': 'base_54k'
}
DEFAULT_BASE = 'base'

# =============================================================================
# RECLASSIFICATION ANALYSIS
# =============================================================================

def load_for_reclassification(data_source):
    """Loads and processes data specifically for the reclassification analysis."""
    output_pkl = os.path.join(RESULTS_DIR, f"tabular_{data_source}_test.pkl")
    try:
        df = pd.read_pickle(output_pkl)
        df['model_name'] = df['model_name'].map(model_translate)
        df.dropna(subset=['model_name'], inplace=True)
    except FileNotFoundError:
        print(f"ERROR: Reclassification analysis - Results file not found at {output_pkl}.")
        return None
    
    processed_data = {}
    base_names = [name for name in df['model_name'].unique() if 'base' in name]
    if not base_names: return None
    base_models_data = {name: np.concatenate(df[df['model_name'] == name]['y_pred_binary'].values[0]) for name in base_names}
    df_to_process = df[~df['model_name'].str.contains('base')]
    for _, model_row in df_to_process.iterrows():
        model_name = model_row['model_name']
        base_to_use = model_base_map.get(model_name, DEFAULT_BASE)
        if base_to_use not in base_models_data: continue
        y_true_flat = base_models_data[base_to_use]
        y_pred_flat = np.concatenate(model_row['y_pred_binary'])
        if len(y_true_flat) == len(y_pred_flat):
            processed_data[model_name] = {'y_true': y_true_flat, 'y_pred': y_pred_flat}
    return processed_data

def display_reclassification_report(results_data):
    """Displays the reclassification report in the notebook."""
    table_rows = []
    for result in results_data:
        model_name = result['model_name']
        rate1, reclassified1, missed1 = result['intraop_to_preop'].values()
        rate2, reclassified2, missed2 = result['preop_to_combined'].values()
        cell1 = f"<td>{rate1:.1f}% ({reclassified1:.1f} / {missed1:.1f})</td>"
        cell2 = f"<td>{rate2:.1f}% ({reclassified2:.1f} / {missed2:.1f})</td>"
        table_rows.append(f"<tr><td><strong>{model_name}</strong></td>{cell1}{cell2}</tr>")

    table_body_html = "\n".join(table_rows)
    table_html = f"""
    <table class="results-table">
        <thead><tr><th>Model</th><th>Correction Rate: Intraop → Preop</th><th>Correction Rate: Preop → Combined</th></tr></thead>
        <tbody>{table_body_html}</tbody>
    </table>"""
    display_html_with_widget("Patient Reclassification Analysis", "The table below shows the correction rate of initially missed positive cases (false negatives).", table_html)

def analyze_patient_reclassification():
    """Performs and displays the reclassification analysis."""
    print("\n" + "="*80 + "\nSTARTING PATIENT RECLASSIFICATION ANALYSIS\n" + "="*80)
    sources = ["intraop", "preop", "combined"]
    all_data = {source: load_for_reclassification(source) for source in sources}
    if any(data is None for data in all_data.values()): return

    model_sets = [set(data.keys()) for data in all_data.values()]
    common_models = sorted(list(set.intersection(*model_sets)))
    if not common_models: 
        print("No models found in common across all datasets.")
        return

    print(f"Found {len(common_models)} models common to all datasets.")
    
    results_for_html = []
    for model_name in common_models:
        print(f"\n--- Analyzing Model: {model_name} ---")
        model_results = {'model_name': model_name}
        for source_before, source_after in [("intraop", "preop"), ("preop", "combined")]:
            data_before = all_data[source_before][model_name]
            data_after = all_data[source_after][model_name]
            y_true, pred_before, pred_after = data_before['y_true'], data_before['y_pred'], data_after['y_pred']
            
            was_fn = (y_true == 1) & (pred_before == 0)
            is_tp_now = (y_true == 1) & (pred_after == 1)
            
            total_reclassified = np.sum(was_fn & is_tp_now)
            total_fn = np.sum(was_fn)
            
            avg_reclassified = total_reclassified / N_FOLDS
            avg_false_negatives = total_fn / N_FOLDS
            
            rate = (total_reclassified / total_fn) * 100 if total_fn > 0 else 0.0
            
            print(
                f"Going from {source_before.upper()} to {source_after.upper()}:\n"
                f"  The model correctly reclassified an average of {avg_reclassified:.1f} patients.\n"
                f"  This represents a {rate:.1f}% correction rate of the "
                f"{avg_false_negatives:.1f} initially missed positive cases per fold."
            )

            model_results[f"{source_before}_to_{source_after}"] = {
                'rate': rate, 'reclassified': avg_reclassified, 'missed': avg_false_negatives
            }
        results_for_html.append(model_results)
    
    # Display the final HTML table after printing all calculations
    display_reclassification_report(results_for_html)

# =============================================================================
# GENERIC HTML DISPLAY WIDGET
# =============================================================================

def display_html_with_widget(title, description, table_html):
    """A generic function to display any HTML table with a collapsible code widget."""
    style = """
    <style>
        .results-table { border-collapse: collapse; width: 95%; margin-top: 15px; font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif; font-size: 14px; }
        .results-table th, .results-table td { border: 1px solid #dee2e6; padding: 10px; text-align: center; }
        .results-table thead th { background-color: #e9ecef; color: #495057; }
        .results-table tbody tr:nth-child(even) { background-color: #f8f9fa; }
        .results-table .model-name { text-align: left; font-weight: bold; }
        .widget-container { margin-top: 20px; }
        .toggle-link { color: #007bff; text-decoration: none; font-size: 14px; cursor: pointer; }
        .toggle-link:hover { text-decoration: underline; }
        .code-block { display: none; background-color: #f1f1f1; border: 1px solid #ccc; padding: 10px; margin-top: 10px; border-radius: 4px; white-space: pre-wrap; word-wrap: break-word; }
    </style>"""
    
    unique_id = "widget_" + os.urandom(8).hex()
    escaped_html = html.escape(table_html)
    
    full_html = f"""
    <div class="report-section">
        <h3>{title}</h3>
        <p>{description}</p>
        {style}
        {table_html}
        <div class="widget-container">
            <a class="toggle-link" onclick="var x = document.getElementById('{unique_id}'); x.style.display = (x.style.display === 'none' ? 'block' : 'none');">
                Show/Hide HTML Source
            </a>
            <pre id="{unique_id}" class="code-block"><code>{escaped_html}</code></pre>
        </div>
    </div>
    """
    display(HTML(full_html))

# =============================================================================
# MAIN EXECUTION BLOCK
# =============================================================================

def run_all_analyses():
    """Runs all analysis functions to generate a complete report."""
    analyze_patient_reclassification()
    print("\n" + "="*80 + "\nRECLASSIFICATION ANALYSIS COMPLETE\n" + "="*80)

if __name__ == "__main__":
    # In a script, you would call this.
    run_all_analyses()

# To run this in a Jupyter cell, you would call the main function directly:
# run_all_analyses()



STARTING PATIENT RECLASSIFICATION ANALYSIS
Found 7 models common to all datasets.

--- Analyzing Model: AutoGluon ---
Going from INTRAOP to PREOP:
  The model correctly reclassified an average of 80.1 patients.
  This represents a 54.2% correction rate of the 147.9 initially missed positive cases per fold.
Going from PREOP to COMBINED:
  The model correctly reclassified an average of 10.3 patients.
  This represents a 10.4% correction rate of the 98.7 initially missed positive cases per fold.

--- Analyzing Model: GBT ---
Going from INTRAOP to PREOP:
  The model correctly reclassified an average of 92.0 patients.
  This represents a 41.2% correction rate of the 223.2 initially missed positive cases per fold.
Going from PREOP to COMBINED:
  The model correctly reclassified an average of 7.9 patients.
  This represents a 3.9% correction rate of the 204.2 initially missed positive cases per fold.

--- Analyzing Model: KNN ---
Going from INTRAOP to PREOP:
  The model correctly reclassifie

Model,Correction Rate: Intraop → Preop,Correction Rate: Preop → Combined
AutoGluon,54.2% (80.1 / 147.9),10.4% (10.3 / 98.7)
GBT,41.2% (92.0 / 223.2),3.9% (7.9 / 204.2)
KNN,12.6% (69.1 / 546.8),0.0% (0.0 / 477.7)
Logistic Regression,56.9% (90.8 / 159.6),15.0% (16.0 / 107.0)
MLP,58.8% (94.0 / 159.8),21.0% (21.4 / 102.0)
Random Forest,55.9% (230.8 / 413.2),2.3% (4.9 / 209.6)
SVM (Linear),56.3% (89.2 / 158.4),19.7% (21.3 / 108.3)



RECLASSIFICATION ANALYSIS COMPLETE


# Bootstrap CI and DCA

In [7]:
# =============================================================================
# CLINICAL UTILITY ANALYSIS (BOOTSTRAP CI & DECISION-CURVE ANALYSIS)
#
# This cell orchestrates the new analyses by:
# 1. Importing the required functions from the new .py modules.
# 2. Looping through each data source (preop, intraop, combined).
# 3. Loading the corresponding CALIBRATED results file.
# 4. For each model, it prepares the data and calls the bootstrap and DCA functions.
# 5. Aggregates all bootstrap results and saves them to 'metrics_ci.csv'.
# =============================================================================

import pandas as pd
import numpy as np
import os
import pickle

# 1. Import the new analysis functions
from bootstrap_metrics import bootstrap_metrics
from decision_curve import generate_dca_plot

# 2. Configuration
ALL_DATA_SOURCES = ["preop", "intraop", "combined"]
RESULTS_DIR = '/home/server/Projects/data/AKI/results/'
DCA_FIGURES_DIR = 'figures/dca' # Define the new save directory for DCA outputs

# --- Set bootstrap repetitions ---
# Standard value for publication is 1000.
# Lower this value (e.g., to 100) for faster development runs.
BOOTSTRAP_REPS = 1000 

DEFAULT_BASE = 'base'
MODEL_BASE_MAP = {
    'LSTM': 'base_54k',
    'Hybrid (MLP + LSTM)': 'base_54k'
}

# Initialize a list to store all bootstrap results
all_bootstrap_results = []

print(f"{'='*80}\nSTARTING BOOTSTRAP AND DECISION-CURVE ANALYSIS (B={BOOTSTRAP_REPS})\n{'='*80}")

# 3. Outer Loop: Iterate through data sources
for data_source in ALL_DATA_SOURCES:
    print(f"\n{'~'*25} PROCESSING: {data_source.upper()} {'~'*25}\n")
    
    # Construct path to the CALIBRATED results file
    input_pkl = os.path.join(RESULTS_DIR, f"tabular_{data_source}_test_calibrated.pkl")
    
    try:
        with open(input_pkl, 'rb') as f:
            df_calibrated = pickle.load(f)
    except FileNotFoundError:
        print(f"ERROR: Calibrated results file not found at {input_pkl}. Skipping.")
        continue

    # Separate base (ground truth) and predictive models
    df_base = df_calibrated[df_calibrated['model_name'].str.contains('base', na=False)].copy()
    df_models = df_calibrated[~df_calibrated['model_name'].str.contains('base', na=False)].copy()

    # 4. Inner Loop: Iterate through each model
    for _, model_row in df_models.iterrows():
        model_name = model_row['model_name']
        print(f"Analyzing model: {model_name}")

        # 5. Data Preparation
        base_to_use = MODEL_BASE_MAP.get(model_name, DEFAULT_BASE)
        try:
            # Extract lists of arrays for y_true, p_calib, and tau_star
            y_true_folds = df_base[df_base['model_name'] == base_to_use]['y_pred_binary'].iloc[0]
            p_calib_folds = model_row['y_prob']
            tau_star = model_row['threshold']

            # Flatten the lists of arrays into single 1D vectors
            y_true_flat = np.concatenate(y_true_folds)
            p_calib_flat = np.concatenate(p_calib_folds)
            
            # Ensure data is consistent
            if len(y_true_flat) != len(p_calib_flat):
                print(f"  -> WARNING: Mismatched length for y_true and p_calib. Skipping.")
                continue

        except (IndexError, KeyError) as e:
            print(f"  -> WARNING: Could not find base model '{base_to_use}' or required data for '{model_name}'. Skipping. Error: {e}")
            continue

        # 6. Execute Bootstrap CI Calculation
        bootstrap_results = bootstrap_metrics(y_true_flat, p_calib_flat, tau_star, B=BOOTSTRAP_REPS)
        
        # Store results in a structured format
        for metric_name, values in bootstrap_results.items():
            point_est, lower_ci, upper_ci = values
            all_bootstrap_results.append({
                'data_source': data_source,
                'model_name': model_name,
                'metric': metric_name,
                'point_estimate': point_est,
                'lower_95_ci': lower_ci,
                'upper_95_ci': upper_ci
            })
        print("  -> Completed bootstrap CI calculation.")

        # 7. Execute Decision-Curve Analysis
        generate_dca_plot(
            y_true=y_true_flat,
            p_calib=p_calib_flat,
            tau_star=tau_star,
            data_source_name=data_source,
            model_name=model_name,
            save_dir=DCA_FIGURES_DIR,
            B=BOOTSTRAP_REPS
        )


# 8. Final Aggregation and Output
# Convert the list of results into a final DataFrame
final_results_df = pd.DataFrame(all_bootstrap_results)

# Save the DataFrame to a CSV file
output_csv_path = 'metrics_ci.csv'
final_results_df.to_csv(output_csv_path, index=False)

print(f"\n{'='*80}\nANALYSIS COMPLETE\n{'='*80}")
print(f"Successfully saved all bootstrap confidence intervals to: {output_csv_path}")
print("\nFinal Results Preview:")
display(final_results_df.head())


STARTING BOOTSTRAP AND DECISION-CURVE ANALYSIS (B=1000)

~~~~~~~~~~~~~~~~~~~~~~~~~ PROCESSING: PREOP ~~~~~~~~~~~~~~~~~~~~~~~~~

Analyzing model: Logistic Regression
  -> Completed bootstrap CI calculation.
  -> Saved DCA plot to: figures/dca/dca_curve_preop_Logistic_Regression.png
  -> Saved DCA data to: figures/dca/dca_table_preop_Logistic_Regression.csv
Analyzing model: GBT
  -> Completed bootstrap CI calculation.
  -> Saved DCA plot to: figures/dca/dca_curve_preop_GBT.png
  -> Saved DCA data to: figures/dca/dca_table_preop_GBT.csv
Analyzing model: Random Forest
  -> Completed bootstrap CI calculation.
  -> Saved DCA plot to: figures/dca/dca_curve_preop_Random_Forest.png
  -> Saved DCA data to: figures/dca/dca_table_preop_Random_Forest.csv
Analyzing model: SVM (Linear)
  -> Completed bootstrap CI calculation.
  -> Saved DCA plot to: figures/dca/dca_curve_preop_SVM_Linear.png
  -> Saved DCA data to: figures/dca/dca_table_preop_SVM_Linear.csv
Analyzing model: MLP
  -> Completed bootstr

,data_source,model_name,metric,point_estimate,lower_95_ci,upper_95_ci
0,preop,Logistic Regression,auroc,0.911400,0.908674,0.914068
1,preop,Logistic Regression,auprc,0.526227,0.517332,0.535033
2,preop,Logistic Regression,f2,0.577690,0.571446,0.584188
3,preop,Logistic Regression,brier,0.028542,0.028065,0.028997
4,preop,Logistic Regression,calib_int,NaN,NaN,NaN


In [2]:
# =============================================================================
# CELL 3: CROSS-DATASET DECISION-CURVE ANALYSIS COMPARISON
#
# This cell generates comparative DCA plots to assess the value of adding
# more data. It creates one plot per model, showing the DCA curve for that
# model on the 'preop', 'intraop', and 'combined' datasets.
# =============================================================================

import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from decision_curve import calculate_net_benefit_for_thresholds # Import the newly public helper

# --- CONFIGURATION (Added to make cell self-contained) ---
ALL_DATA_SOURCES = ["preop", "intraop", "combined"]
RESULTS_DIR = '/home/server/Projects/data/AKI/results/'
DCA_FIGURES_DIR = 'figures/dca'
MODEL_BASE_MAP = {
    'LSTM': 'base_54k',
    'Hybrid (MLP + LSTM)': 'base_54k'
}
DEFAULT_BASE = 'base'
# ---------------------------------------------------------

print(f"{'='*80}\nSTARTING CROSS-DATASET DCA COMPARISON\n{'='*80}")

# Find all unique model names across all datasets
all_models = set()
for data_source in ALL_DATA_SOURCES:
    input_pkl = os.path.join(RESULTS_DIR, f"tabular_{data_source}_test_calibrated.pkl")
    try:
        df = pd.read_pickle(input_pkl)
        # Filter out base models
        models_in_source = df[~df['model_name'].str.contains('base', na=False)]['model_name'].unique()
        all_models.update(models_in_source)
    except FileNotFoundError:
        continue

# Outer Loop: Iterate through each unique model name
for model_name in sorted(list(all_models)):
    print(f"\n{'~'*25} COMPARING DATA SOURCES FOR: {model_name} {'~'*25}")
    
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(8, 6), dpi=300)
    
    # --- Data Collection for the current model ---
    has_data = False
    
    # Inner Loop: Iterate through data sources to find this model's predictions
    for data_source in ALL_DATA_SOURCES:
        input_pkl = os.path.join(RESULTS_DIR, f"tabular_{data_source}_test_calibrated.pkl")
        try:
            df_calibrated = pd.read_pickle(input_pkl)
            model_row = df_calibrated[df_calibrated['model_name'] == model_name]

            if model_row.empty:
                continue # Model not in this data source

            # Extract data for this source
            df_base = df_calibrated[df_calibrated['model_name'].str.contains('base', na=False)].copy()
            base_to_use = MODEL_BASE_MAP.get(model_name, DEFAULT_BASE)
            
            y_true_folds = df_base[df_base['model_name'] == base_to_use]['y_pred_binary'].iloc[0]
            p_calib_folds = model_row['y_prob'].iloc[0]

            y_true_flat = np.concatenate(y_true_folds)
            p_calib_flat = np.concatenate(p_calib_folds)

            # Calculate Net Benefit for this data source
            pt_grid = np.arange(0.01, 0.305, 0.005)
            nb_model = calculate_net_benefit_for_thresholds(y_true_flat, p_calib_flat, pt_grid)
            
            # Plot the curve for this data source
            ax.plot(pt_grid * 100, nb_model, lw=2, label=f'{data_source.capitalize()} Data')
            has_data = True

        except (FileNotFoundError, IndexError, KeyError):
            continue
            
    # --- Finalize and Save the Comparative Plot ---
    if has_data:
        # Add reference lines (calculated from a representative dataset, e.g., 'combined')
        try:
            combined_pkl = os.path.join(RESULTS_DIR, "tabular_combined_test_calibrated.pkl")
            df_comb = pd.read_pickle(combined_pkl)
            df_base_comb = df_comb[df_comb['model_name'].str.contains('base', na=False)].copy()
            y_true_comb = np.concatenate(df_base_comb.iloc[0]['y_pred_binary'])
            
            prevalence = y_true_comb.mean()
            nb_all = prevalence - (1 - prevalence) * (pt_grid / (1 - pt_grid))
            
            ax.plot(pt_grid * 100, nb_all, color='black', lw=1.5, ls='--', label='Treat-All Strategy')
            ax.axhline(0, color='grey', lw=1.5, ls='-', label='Treat-None Strategy')
        except (FileNotFoundError, IndexError):
            print("  -> Could not add reference lines; 'combined' data not found.")

        ax.set_title(f'DCA: Impact of Data Source on {model_name}', fontsize=16)
        ax.set_xlabel('Risk Threshold Probability (%)', fontsize=12)
        ax.set_ylabel('Net Benefit', fontsize=12)
        ax.set_xlim(1, 30)
        ax.set_ylim(-0.05, 0.2)
        ax.legend(loc='upper right', fontsize=10)
        plt.tight_layout()

        safe_model_name = model_name.replace(" ", "_").replace("(", "").replace(")", "")
        save_path = os.path.join(DCA_FIGURES_DIR, f'dca_datasource_comparison_{safe_model_name}.png')
        fig.savefig(save_path)
        plt.close(fig)
        print(f"  -> Saved comparison plot to: {save_path}")
    else:
        plt.close(fig)
        print(f"  -> No data found for model '{model_name}'. Skipping plot.")

print(f"\n{'='*80}\nCROSS-DATASET DCA COMPARISON COMPLETE\n{'='*80}")


STARTING CROSS-DATASET DCA COMPARISON

~~~~~~~~~~~~~~~~~~~~~~~~~ COMPARING DATA SOURCES FOR: ASA Rule ~~~~~~~~~~~~~~~~~~~~~~~~~
  -> Saved comparison plot to: figures/dca/dca_datasource_comparison_ASA_Rule.png

~~~~~~~~~~~~~~~~~~~~~~~~~ COMPARING DATA SOURCES FOR: AutoGluon ~~~~~~~~~~~~~~~~~~~~~~~~~
  -> Saved comparison plot to: figures/dca/dca_datasource_comparison_AutoGluon.png

~~~~~~~~~~~~~~~~~~~~~~~~~ COMPARING DATA SOURCES FOR: GBT ~~~~~~~~~~~~~~~~~~~~~~~~~
  -> Saved comparison plot to: figures/dca/dca_datasource_comparison_GBT.png

~~~~~~~~~~~~~~~~~~~~~~~~~ COMPARING DATA SOURCES FOR: Hybrid (MLP + LSTM) ~~~~~~~~~~~~~~~~~~~~~~~~~
  -> Saved comparison plot to: figures/dca/dca_datasource_comparison_Hybrid_MLP_+_LSTM.png

~~~~~~~~~~~~~~~~~~~~~~~~~ COMPARING DATA SOURCES FOR: KNN ~~~~~~~~~~~~~~~~~~~~~~~~~
  -> Saved comparison plot to: figures/dca/dca_datasource_comparison_KNN.png

~~~~~~~~~~~~~~~~~~~~~~~~~ COMPARING DATA SOURCES FOR: LSTM ~~~~~~~~~~~~~~~~~~~~~~~~~
  -> Saved com